In [1]:
import os
import sys
import warnings
from itertools import groupby
from multiprocessing import Pool

import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from scipy.spatial.distance import cdist, squareform, cosine, euclidean, cityblock
from sklearn.manifold import trustworthiness
from umap import UMAP

import torch

import h5py

sys.dont_write_bytecode = True
np.set_printoptions(precision=6, suppress=True)

from my_library import Database, Metrics, ESM_Representations, gzip_tensor, neighbor_joining, cophenetic_distmat, silhouette

RANDOM_SEED = 420

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
/Users/olivierdennler/miniconda/envs/chumby/lib/python3.8/site-packages/umap/distances.py:1053: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/olivierdennler/miniconda/envs/chumby/lib/python3.8/site-packages/umap/distances.py:1061: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/referen

## Load Uniprot ProtT5 embeddings

Documentation: https://www.uniprot.org/help/embeddings

Download: https://www.uniprot.org/help/downloads#embeddings

In [2]:
# Init a dict: {uniprot_id : embedding}
dict_id_emb = {}

# If we use 'per protein' embeddings files, we don't need to normalize size
with h5py.File('/Users/olivierdennler/Documents/data/SLI_2023/embeddings_hs/hs_uniprot_ProtT5_per-protein.h5') as h5:
        for uniprot_id, embedding in h5.items():
            # Extract protein embedding (fixed size) as 1D array
            emb = np.array(embedding)
            dict_id_emb[uniprot_id] = emb


In [ ]:
# To compute cosine distances for 2 embeddings
# dist_cosine = cosine(emb,emb)

## Save the distances in a table file (as pair features)

In [21]:
def ts_ss(x1, x2):
    """
    Stands for triangle area similarity (TS) and sector area similarity (SS)
    For more information: https://github.com/taki0112/Vector_Similarity
    """
    #  ensures that x1 and x2 are treated as 2D arrays even if they're originally 1D (added by Chat-GPT)
    if x1.ndim == 1:
        x1 = x1[np.newaxis, :]
    if x2.ndim == 1:
        x2 = x2[np.newaxis, :]

    x1_norm = np.linalg.norm(x1, axis=-1)[:,np.newaxis]
    x2_norm = np.linalg.norm(x2, axis=-1)[:,np.newaxis]
    x_dot = x1_norm @ x2_norm.T

    ### cosine similarity
    cosine_sim = 1 - cdist(x1, x2, metric='cosine')
    cosine_sim[cosine_sim != cosine_sim] = 0
    cosine_sim = np.clip(cosine_sim, -1, 1, out=cosine_sim)

    ### euclidean_distance
    euclidean_dist = cdist(x1, x2, metric='euclidean')

    ### triangle_area_similarity
    theta = np.arccos(cosine_sim) + np.radians(10)
    triangle_similarity = (x_dot * np.abs(np.sin(theta))) / 2

    ### sectors area similarity
    magnitude_diff = np.abs(x1_norm - x2_norm.T)
    ed_plus_md = euclidean_dist + magnitude_diff
    sector_similarity =  ed_plus_md * ed_plus_md * theta * np.pi / 360

    ### hybridize
    similarity = triangle_similarity * sector_similarity
    return similarity[0][0]

In [22]:
# Use uniprot gene mapping
# Efficient data loading
cols_to_use_uniprot = ['gene', 'uniprot']  # Specify the columns you need
df_gene_uniprot = pd.read_table('/Users/olivierdennler/Documents/data/SLI_2023/map_gene_uniprot_ens111.csv', sep=',', usecols=cols_to_use_uniprot, low_memory=False)

cols_to_use_pairs = ['sorted_gene_pair', 'A1_entrez', 'A2_entrez']  # Specify the columns you need
#gene_pairs_full = pd.read_csv('/Users/olivierdennler/Documents/data/SLI_2023/S8_DeKegel.csv', usecols=cols_to_use_pairs)
gene_pairs_full = pd.read_csv('/Users/olivierdennler/Documents/data/SLI_2023/paralog_SL_prediction-master/local_data/processed/paralog_features/all_features.csv', usecols=cols_to_use_pairs)

# Convert gene_pairs_full to a set for faster lookup
gene_pairs_set = set(gene_pairs_full['sorted_gene_pair'])

# Initialize DataFrame only once
df = pd.DataFrame()

# Iterate directly on gene pairs of interest
for gene_pair in gene_pairs_set:

    # New version using uniprot gene mapping file
    # We can directly use gene pair
    A1, A2 = gene_pair.split('_')
    # Iterate on gene and get their uniprots
    for gene_id_i in [A1,A2]:
        uniprot_id_i = str(df_gene_uniprot.loc[df_gene_uniprot['gene'] == gene_id_i, 'uniprot'].squeeze())
        if gene_id_i == A1: 
            paralog = 'A1'
            uniprot_i = uniprot_id_i
        elif gene_id_i == A2: 
            paralog = 'A2'
            uniprot_j = uniprot_id_i

    if uniprot_i is not None and uniprot_j is not None and not pd.isna(uniprot_i) and not pd.isna(uniprot_j):

        # Cases if multiple uniprot for one symbol
        if len(uniprot_i.split('|')) > 1:
            uniprot_i = uniprot_i.split('|')
        else:
            uniprot_i = [uniprot_i]
        for i in uniprot_i:
            if i in dict_id_emb:
                uniprot_i = i
                break 
        if len(uniprot_j.split('|')) > 1:
            uniprot_j = uniprot_j.split('|')
        else:
            uniprot_j = [uniprot_j]
        for j in uniprot_j:
            if j in dict_id_emb:
                uniprot_j = j
                break 

        if isinstance(uniprot_i, str) and isinstance(uniprot_j, str):
            # Get their embeddings
            emb_i = dict_id_emb[uniprot_i]
            emb_j = dict_id_emb[uniprot_j]

            # Compute cosine distance
            dist_cosine_i_j = cosine(emb_i,emb_j)
            # Compute euclidean distance
            dist_euclidean_i_j = euclidean(emb_i,emb_j)
            # Compute manhattan distance
            dist_manhattan_i_j = cityblock(emb_i,emb_j)
            # Compute ts_ss distance
            dist_ts_ss_i_j = ts_ss(emb_i,emb_j)

            # Add row to dataframe
            row = pd.DataFrame({
                        'sorted_gene_pair': [gene_pair], 
                        'ProtT5_per-protein_cosine': [dist_cosine_i_j],
                        'ProtT5_per-protein_euclidean': [dist_euclidean_i_j],
                        'ProtT5_per-protein_manhattan': [dist_manhattan_i_j],
                        'ProtT5_per-protein_ts_ss': [dist_ts_ss_i_j]
                        })
            df = pd.concat([df, row], ignore_index = True) 
            
        else:
            print(f'=> No distance for {gene_pair}')
            if isinstance(uniprot_i, list):
                print(f'{uniprot_i[0]} is not in Uniprot Embedding file ... (the entry may have been deleted from uniprot)')
            if isinstance(uniprot_j, list):
                print(f'{uniprot_j[0]} is not in Uniprot Embedding file ... (the entry may have been deleted from uniprot)')

# Remove duplicates after all merges are done
df = df.drop_duplicates(subset=['sorted_gene_pair'])

df


=> No distance for GOLGA8B_GOLGA8G
P0DX52 is not in Uniprot Embedding file ... (the entry may have been deleted from uniprot)
=> No distance for CNTNAP3C_MEP1A
nan is not in Uniprot Embedding file ... (the entry may have been deleted from uniprot)
=> No distance for GOLGA8G_GOLGA8S
P0DX52 is not in Uniprot Embedding file ... (the entry may have been deleted from uniprot)
=> No distance for GOLGA8B_GOLGA8F
P0DX52 is not in Uniprot Embedding file ... (the entry may have been deleted from uniprot)
=> No distance for GOLGA6D_GOLGA8G
P0DX52 is not in Uniprot Embedding file ... (the entry may have been deleted from uniprot)
=> No distance for WFDC6_WFIKKN1
Q9BQY6 is not in Uniprot Embedding file ... (the entry may have been deleted from uniprot)
=> No distance for SPINT1_WFDC6
Q9BQY6 is not in Uniprot Embedding file ... (the entry may have been deleted from uniprot)
=> No distance for CNTNAP1_CNTNAP3C
nan is not in Uniprot Embedding file ... (the entry may have been deleted from uniprot)
=> 

,sorted_gene_pair,ProtT5_per-protein_cosine,ProtT5_per-protein_euclidean,ProtT5_per-protein_manhattan,ProtT5_per-protein_ts_ss
0,OR1M1_OR4C3,0.065430,0.506348,12.796875,0.000800
1,ZNF235_ZNF600,0.119141,0.973145,24.359375,0.011317
2,USP50_USP7,0.606934,1.439453,36.187500,0.020899
3,KRT2_KRT3,0.021484,0.372803,9.406250,0.000382
4,DNAH9_DYNC2H1,0.114258,0.628418,15.679688,0.001400
...,...,...,...,...,...
123736,CHRNB1_GABRR3,0.174805,0.640625,16.406250,0.001208
123737,ZNF300_ZNF790,0.067383,0.670410,17.000000,0.002777
123738,DDX46_DDX6,0.236816,0.788086,19.671875,0.002633
123739,SEPTIN10_SEPTIN14,0.035156,0.276855,7.035156,0.000079


In [24]:
# Write dataset in csv file
df.to_csv('ens111_human_embeddings_ProtT5_distances.csv', sep=',', index=False)